In [42]:
import sys
import uproot
import os
import numpy as np
import pandas as pd
from numpy.linalg import eig
from sklearn.decomposition import PCA

In [43]:
part =  "gamma" # "electron" "muon" "pion_c"
path = "/data/user/adipilat/ParticleID/genEvts/"
unpad_path = "/data/user/adipilat/ParticleID/genEvts/new_datasets/unpadded/"
pad_path = "/data/user/adipilat/ParticleID/genEvts/new_datasets/padded/"
dir_ = "ana"
tree = "hgc"
max_perlayer = 10
number_layers = 50

In [44]:
variableName = [
            'event',
            'cluster2d_layer',
            'cluster2d_energy',
            'cluster2d_eta',
            'cluster2d_phi',
            'cluster2d_pt',
            'cluster2d_x',
            'cluster2d_y',
            'cluster2d_z',
            'cluster2d_nhitCore',
            'cluster2d_nhitAll',
            'gen_energy',
            'gen_pdgid',
            'gen_daughters',
            'gen_phi',
            'gen_eta',
            'cluster2d_best_cpPdg',
            'tracksterEM_clusters',
            'tracksterMIP_clusters',
            'tracksterHAD_clusters'
            ]
newVars =["event","tracksterEM","tracksterMIP","tracksterHAD","x","y","z","r","layer","E","nCore","nHits","id","genDR","gen_phi","gen_eta","phi","eta","genPID","genE","cpID"]

In [45]:
name = "4_" + part + "_new"
file = path + part + "/" + name + ".root"
print("Starting data production for "+ part)

Starting data production for gamma


In [46]:
df = uproot.open(file)[dir_][tree].pandas.df(variableName,flatten=False)


num_events = np.unique(df["event"].values).shape[0]
xs = df["cluster2d_x"].values
ys = df["cluster2d_y"].values
zs = df["cluster2d_z"].values
es = df["cluster2d_energy"].values
ps = df["cluster2d_pt"].values
nh = df["cluster2d_nhitAll"].values
nc = df["cluster2d_nhitCore"].values
ll = df["cluster2d_layer"].values
ee = df["event"].values
    
sizes = [x.shape[0] for x in xs]
indices = [np.full((a[0]),a[1]) for a in zip(sizes,range(len(sizes)))]

cphi = df["cluster2d_phi"].values
ceta = df["cluster2d_eta"].values
gpid = df["gen_pdgid"].values
gen = df["gen_energy"].values

gphi = [np.full((a[0]),a[1]) for a in zip(sizes,df["gen_phi"].values)]
geta = [np.full((a[0]),a[1]) for a in zip(sizes,df["gen_eta"].values)]
gpid = [np.full((a[0]),a[1]) for a in zip(sizes,df["gen_pdgid"].values)]
gen = [np.full((a[0]),a[1]) for a in zip(sizes,df["gen_energy"].values)]

cp = df["cluster2d_best_cpPdg"].values
trEM = df["tracksterEM_clusters"].values
trMIP = df["tracksterMIP_clusters"].values
trHAD = df["tracksterHAD_clusters"].values

In [48]:
idtrlistEM = []
# LayerClusters that don't belong to any Trackster will have TracksterId = 0. Real Tracksters have the TracksterId > 0
for i in range(len(sizes)):
    idtrlistEM.append(np.array([0]*sizes[i]))
idtrlistMIP = idtrlistEM
idtrlistHAD = idtrlistEM

for i in range(len(trEM)):
    for j in range(len(trEM[i])):
        for item in trEM[i][j]:
            idtrlistEM[i][item] = j + 1
for i in range(len(trMIP)):
    for j in range(len(trMIP[i])):
        for item in trMIP[i][j]:
            idtrlistMIP[i][item] = j + 1
for i in range(len(trHAD)):
    for j in range(len(trHAD[i])):
        for item in trHAD[i][j]:
            idtrlistHAD[i][item] = j + 1

idtrEM = np.array(idtrlistEM)
idtrMIP = np.array(idtrlistMIP)
idtrHAD = np.array(idtrlistHAD)


[ True]


/data/user/adipilat/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:25: DeprecationWarning: elementwise != comparison failed; this will raise an error in the future.


In [36]:
rs = [np.sqrt(f[0]**2+f[1]**2) for f in zip(xs,ys)]
drs = [np.sqrt((a[0]-a[1])**2 + (a[2]-a[3])**2) for a in zip(gphi,cphi,geta,ceta)]

In [37]:
XS = np.array([item for sublist in xs for item in sublist])
YS = np.array([item for sublist in ys for item in sublist])
ZS = np.array([item for sublist in zs for item in sublist])
RS = np.array([item for sublist in rs for item in sublist])
LL = np.array([item for sublist in ll for item in sublist])
ES = np.array([item for sublist in es for item in sublist])
NC = np.array([item for sublist in nc for item in sublist])
NH = np.array([item for sublist in nh for item in sublist])
II = np.array([item for sublist in indices for item in sublist])
DRS = np.array([item for sublist in drs for item in sublist])
GPHI = np.array([item for sublist in gphi for item in sublist])
GETA = np.array([item for sublist in geta for item in sublist])
GPID = np.array([item for sublist in gpid for item in sublist])
GEN = np.array([item for sublist in gen for item in sublist])
CPHI = np.array([item for sublist in cphi for item in sublist])
CETA = np.array([item for sublist in ceta for item in sublist])


SS = [np.full((s,),s) for s in sizes]
EE = [np.full((s,),i) for i,s in zip(ee,sizes)]

SS = np.array([item for sublist in SS for item in sublist])
EE = np.array([item for sublist in EE for item in sublist])

CP = np.array([item for sublist in cp for item in sublist])
TREM = np.array([item for sublist in idtrEM for item in sublist])
TRMIP = np.array([item for sublist in idtrMIP for item in sublist])
TRHAD = np.array([item for sublist in idtrHAD for item in sublist])

datas = np.vstack((EE,TREM,TRMIP,TRHAD,XS,YS,ZS,RS,LL,ES,NC,NH,II,DRS,GPHI,GETA,CPHI,CETA,GPID,GEN,CP)).T

In [38]:
df = pd.DataFrame(datas,columns=newVars)
df = df.sort_values(["event","layer","E"],ascending=[True,True,False]).reset_index(drop=True)

In [39]:
# drop LCs that don't belong to any type of trackster
indexNames = df[(df['tracksterEM'] == 0) & (df['tracksterMIP'] == 0) & (df['tracksterHAD'] == 0)].index
df.drop(indexNames, inplace=True)
df = df.reset_index(drop=True)

In [40]:
df.to_hdf(unpad_path + part + "_aT.h5","data",complevel=0)

In [41]:
df[1000:1100]

,event,tracksterEM,tracksterMIP,tracksterHAD,x,y,z,r,layer,E,...,nHits,id,genDR,gen_phi,gen_eta,phi,eta,genPID,genE,cpID
1000,14.0,1.0,1.0,1.0,61.360443,-86.522018,345.862762,106.071503,17.0,0.140241,...,4.0,11.0,0.041402,-0.985656,1.871201,-0.953930,1.897801,22.0,135.015259,22.0
1001,14.0,1.0,1.0,1.0,61.184258,-92.343681,346.807251,110.773956,18.0,1.556914,...,18.0,11.0,0.012195,-0.985656,1.871201,-0.985634,1.859007,22.0,135.015259,22.0
1002,14.0,1.0,1.0,1.0,58.862209,-89.176422,346.807251,106.851265,18.0,0.304994,...,5.0,11.0,0.022273,-0.985656,1.871201,-0.987378,1.893407,22.0,135.015259,22.0
1003,14.0,1.0,1.0,1.0,61.219051,-85.318497,346.807251,105.009613,18.0,0.029736,...,2.0,11.0,0.053816,-0.985656,1.871201,-0.948398,1.910035,22.0,135.015259,22.0
1004,14.0,1.0,1.0,1.0,61.184784,-92.711182,348.832764,111.080788,19.0,1.875368,...,30.0,11.0,0.009455,-0.985656,1.871201,-0.987458,1.861919,22.0,135.015259,22.0
1005,14.0,1.0,1.0,1.0,55.633995,-92.975105,348.832764,108.349030,19.0,0.077554,...,3.0,11.0,0.048143,-0.985656,1.871201,-1.031572,1.885672,22.0,135.015259,22.0
1006,14.0,1.0,1.0,1.0,61.295029,-92.528412,349.777252,110.989136,20.0,0.896496,...,18.0,11.0,0.005919,-0.985656,1.871201,-0.985722,1.865283,22.0,135.015259,22.0
1007,14.0,1.0,1.0,1.0,55.531597,-93.056519,349.777252,108.366386,20.0,0.176808,...,4.0,11.0,0.050053,-0.985656,1.871201,-1.032769,1.888102,22.0,135.015259,22.0
1008,14.0,1.0,1.0,1.0,58.605461,-95.863113,349.777252,112.358070,20.0,0.082778,...,2.0,11.0,0.040448,-0.985656,1.871201,-1.022076,1.853605,22.0,135.015259,22.0
1009,14.0,1.0,1.0,1.0,63.348694,-87.213058,349.777252,107.792274,20.0,0.071527,...,2.0,11.0,0.048347,-0.985656,1.871201,-0.942593,1.893177,22.0,135.015259,22.0


In [22]:
#use a different dataset for each type of trackster
dfEM = df.copy()
dfMIP = df.copy()
dfHAD = df.copy()

In [23]:
#drop useless LCs
indexEM = dfEM[dfEM['tracksterEM'] == 0].index
indexMIP = dfMIP[dfMIP['tracksterMIP'] == 0].index
indexHAD = dfHAD[dfHAD['tracksterHAD'] == 0].index

dfEM.drop(indexEM, inplace=True)
dfMIP.drop(indexMIP, inplace=True)
dfHAD.drop(indexHAD, inplace=True)

dfEM = dfEM.reset_index(drop=True)
dfMIP = dfMIP.reset_index(drop=True)
dfHAD = dfHAD.reset_index(drop=True)

del dfEM['tracksterMIP']
del dfEM['tracksterHAD']
del dfMIP['tracksterEM']
del dfMIP['tracksterHAD']
del dfHAD['tracksterEM']
del dfHAD['tracksterMIP']

dfEM = dfEM.rename(columns={'tracksterEM': 'trackster'})
dfMIP = dfMIP.rename(columns={'tracksterMIP': 'trackster'})
dfHAD = dfHAD.rename(columns={'tracksterHAD': 'trackster'})

In [24]:
dflist = [dfEM, dfMIP, dfHAD]

In [25]:
for key in dflist:
    key = key.sort_values(["event","trackster","layer","E"],ascending=[True,True,True,False]).reset_index(drop=True)

In [26]:
#get the tracksters' energy sums, the most energetic trackster id for each event, the index of all LC not belonging to it and drop them
for key in dflist:
    energysums = key.groupby(['event', 'trackster'], as_index=False)['E'].sum()
    maxen = energysums.loc[energysums.groupby('event', as_index=False)['E'].idxmax()]['trackster'].values
    indexNames = []
    
    for i in range(int(key['event'].max())):
        indexNames.append(key[(key['event'] == i+1) & (key['trackster'] != maxen[i])].index)
    
    indices = np.array([item for sublist in indexNames for item in sublist])
    key.drop(indices, inplace=True)
    key = key.reset_index(drop=True)

In [27]:
dfMIP[0:100]

,event,trackster,x,y,z,r,layer,E,nCore,nHits,id,genDR,gen_phi,gen_eta,phi,eta,genPID,genE,cpID
0,1.0,1.0,60.399891,-73.928360,325.072754,95.464912,3.0,1.030484,5.0,5.0,0.0,0.008116,-0.890417,1.945988,-0.885772,1.939333,22.0,62.874428,22.0
1,1.0,1.0,60.698460,-74.749603,326.017303,96.290222,4.0,1.806813,5.0,5.0,0.0,0.012241,-0.890417,1.945988,-0.888767,1.933859,22.0,62.874428,22.0
2,1.0,1.0,61.043133,-75.384644,328.042755,97.000565,5.0,4.278848,11.0,11.0,0.0,0.013241,-0.890417,1.945988,-0.890135,1.932750,22.0,62.874428,22.0
3,1.0,1.0,61.142971,-75.626999,328.987274,97.251770,6.0,5.786702,12.0,12.0,0.0,0.012970,-0.890417,1.945988,-0.890906,1.933026,22.0,62.874428,22.0
4,1.0,1.0,61.438732,-76.029900,331.012726,97.751030,7.0,6.109771,17.0,17.0,0.0,0.012008,-0.890417,1.945988,-0.891144,1.934002,22.0,62.874428,22.0
5,1.0,1.0,58.605446,-74.006523,331.012726,94.401077,7.0,0.072731,2.0,2.0,0.0,0.023974,-0.890417,1.945988,-0.901017,1.967491,22.0,62.874428,22.0
6,1.0,1.0,64.884468,-76.366417,331.012726,100.208900,7.0,0.069516,2.0,2.0,0.0,0.043032,-0.890417,1.945988,-0.866507,1.910209,22.0,62.874428,22.0
7,1.0,1.0,61.617496,-76.328186,331.957245,98.095406,8.0,6.428619,18.0,18.0,0.0,0.012685,-0.890417,1.945988,-0.891638,1.933362,22.0,62.874428,22.0
8,1.0,1.0,61.879799,-76.942612,333.982758,98.738419,9.0,6.560820,17.0,17.0,0.0,0.013412,-0.890417,1.945988,-0.893480,1.932930,22.0,62.874428,22.0
9,1.0,1.0,57.902843,-78.366272,333.982758,97.437218,9.0,0.045406,2.0,2.0,0.0,0.044038,-0.890417,1.945988,-0.934454,1.945658,22.0,62.874428,22.0


In [16]:
#Reassign a trackster number and drop useless stuff
for key in dflist:
    trackster_sizes = key.groupby(["event"]).size().values.tolist()
    trackster_places = np.cumsum(trackster_sizes)
    num_tracksters = len(trackster_sizes)
    track_startes = np.array( [0] + list(trackster_places[:-1]))
    track_finishes = np.array(list(track_startes[1:]) +[len(df)])
    track_id = np.arange(1,num_tracksters+1)
    track_bounds = np.vstack((track_startes,track_finishes)).T
    new_tracks = [[i for j in range(t[1]-t[0])] for i,t in zip(track_id, track_bounds)]
    new_tracks = np.array([item for sublist in new_tracks for item in sublist])
    key['trackster'] = new_tracks
    del key['id']
    del key['event']
    del key['nCore']

ValueError: Length of values does not match length of index

In [ ]:
# compute pca and mean_eta and mean_phi per trackster

for key in dflist:
    means = []
    pca_coordinates = []
    for i in range(1, key.trackster.max()+1):
        pe = np.array([key["phi"][key["trackster"]==i].values, key["eta"][key["trackster"]==i].values])
        en = np.array(key["E"][key["trackster"]==i].values).T

        # compute pca
        pcaVars= np.array([key["x"][key["trackster"]==i].values, key["y"][key["trackster"]==i].values, key["z"][key["trackster"]==i].values]).T  
        pca = PCA()
        pca.fit(pcaVars)
        pca_matrix = pca.transform(pcaVars)    

        temp = [np.average(pe, axis=1, weights=en)]    
        means.append(np.array(temp*trackster_sizes[i-1]))
        pca_coordinates.append(pca_matrix)

    mean_values = np.array([item for sublist in means for item in sublist])
    pca_variables = np.array([item for sublist in pca_coordinates for item in sublist])
    
    key.loc[:,"x_pca"] = pca_variables[:,0]
    key.loc[:,"y_pca"] = pca_variables[:,1]
    key.loc[:,"z_pca"] = pca_variables[:,2]
    key.loc[:,"phi_mean"] = mean_values[:,0]
    key.loc[:,"eta_mean"] = mean_values[:,1]

In [ ]:
for key in dflist:
    theIndex = list(key.groupby(["trackster","layer"]).indices.values())
    theIndex = np.array([item for sublist in theIndex for item in sublist[:min(len(sublist),10)]])
    key = key.iloc[theIndex]

In [ ]:
#Introduce proper indices to copy the old datasets into the padded ones
for key in dflist:
    layer_sizes = key.groupby(["trackster","layer"]).size().values.tolist()
    layer_places = np.cumsum(layer_sizes)
    
    startes = np.array( [0] + list(layer_places[:-1]))
    layers = key["layer"].values[startes]
    ids = key["trackster"].values[startes]
    finishes = np.array(list(startes[1:]) +[len(key)])
    SSS = np.vstack((startes,finishes)).T
    
    hitIds = [[j +(n-1)*max_perlayer + max_perlayer*number_layers*(e-1) for j in range(s[1]-s[0])] for n,s,e in zip(layers,SSS,ids)]
    hitIds = np.array([item for sublist in hitIds for item in sublist])
    
    key.loc[:,"hitIds"] = hitIds
    key = key.set_index(hitIds.astype(int))

In [ ]:
dataset_names = {0:'EM', 1:'MIP', 2:'HAD'}

In [ ]:
#Create the big mask and copy the old dataset in it to have to padded one
for i, key in enumerate(dflist):
    num_tracksters = key.trackster.max()    
    
    bigMask = np.zeros((num_tracksters*number_layers*max_perlayer,len(key.columns)))
    bigDF = pd.DataFrame(bigMask,columns=key.columns)
    
    fakeHit = [ [(i*max_perlayer + j) for j in range(max_perlayer)] for i in range(number_layers*num_tracksters)]
    fakeHit = np.array([item for sublist in fakeHit for item in sublist])
    
    fakeLayer = [ np.full(max_perlayer,i) for j in range(1,num_tracksters+1) for i in range(1,number_layers+1)]
    fakeLayer = np.array([item for sublist in fakeLayer for item in sublist])    
    
    fakeTrackster = [ np.full(max_perlayer*number_layers,i) for i in range(1,num_tracksters+1)]
    fakeTrackster = np.array([item for sublist in fakeTrackster for item in sublist])  
    
    bigDF["layer"] = fakeLayer
    bigDF["trackster"] = fakeTrackster
    bigDF["hitIds"] = fakeHit
    
    bigDF.iloc[key.index] = key
    del bigDF['hitIds']
    
    bigDF.to_hdf(pad_path + part + "_aT" + dataset_names[i] + "Padded.h5","data",complevel=0)